In [ ]:
# ===== Модули Обновлено =====

from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.rt_detr import RT_DETR

import airgen
import numpy as np
import rerun as rr
import cv2
import time
from math import cos, sin, pi

# Инициализация моделей
nav_inspect = ObjectInspect()
nav_visualservoing = VisualServoing()
detection_rt_detr = RT_DETR(use_local=True)  # Инициализация модели детекции

In [ ]:
# ===== Подключение к дрону и взлёт =====

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
# ===== Взлет дрона =====
client.takeoffAsync().join()

# ===== Посадка дрона =====
# client.landAsync().join()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [ ]:
# ===== Создание клиента сцены =====

scene_client = airgen.VehicleClient()
scene_objects = scene_client.simListSceneObjects()
cameras = [obj for obj in scene_objects if 'camera' in obj.lower()]

print("Available cameras:")
for camera in cameras:
    print(camera)

Available cameras:
PlayerCameraManager_2147482464
CameraManagerActor
ExternalCamera
BP_PIPCamera_C_2147482424
BP_PIPCamera_C_2147482412
BP_PIPCamera_C_2147482400
BP_PIPCamera_C_2147482388
BP_PIPCamera_C_2147482376


In [ ]:
# ===== Получение данных LiDAR + визуализация =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)
rr.log('lidar/points', rr.Points3D(points))

In [ ]:
# ===== Получение и логирование изображения с камеры =====

image_data, _ = client.getImages("front_center", [airgen.ImageType.Scene])[0]
rgb_image = cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR)

In [ ]:
# ===== Получение и логирование GPS =====

gps_data = client.getGpsData()
geo = gps_data.gnss.geo_point
gps_position = np.array([[geo.latitude, geo.longitude, geo.altitude]], dtype=np.float32)
rr.log("drone/gps", rr.Points3D(gps_position))

In [ ]:
# Инициализация модели детекции
detection_rt_detr = RT_DETR(use_local=True)

# ===== Получение и логирование изображения с камеры =====
image_data, _ = client.getImages("front_center", [airgen.ImageType.Scene])[0]
rgb_image = cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR)  # Преобразование в BGR для OpenCV

# ===== Распознавание препятствий =====
result = detection_rt_detr.run(input=rgb_image.copy(), confidence_threshold=0.3)  # Снижение порога доверия

# Проверяем, есть ли объекты в результате детекции
if isinstance(result, dict) and 'boxes' in result and len(result['boxes']) > 0:
    boxes = result['boxes']
    labels = result['labels']
    
    image_with_boxes = rgb_image.copy()

    for i, box in enumerate(boxes):
        x, y, w, h = box
        cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image_with_boxes, f"Object {i+1}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        rr.log(
            f"drone/obstacle_{i}",
            rr.Boxes2D(np.array([[x, y, w, h]]), labels=["Obstacle"])
        )

    rr.log("drone/camera", rr.Image(image_with_boxes))

else:
    print("❗ Объекты не обнаружены.")



❗ Объекты не обнаружены.
